In [19]:
## Original code
## https://colab.research.google.com/drive/1SRGYNIyMn1gu7W7TDyS9eHKQmKjpFV-A#scrollTo=qRYUqOgpzWT2

In [20]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

In [21]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [22]:
ticker_data = dataset.copy()

In [23]:
dataF = ticker_data[ticker_data.index > '2020-11-01 00:00:00']

In [24]:
grid_distance = 5
midprice = 1220

In [25]:
def generate_grid(midprice, grid_distance, grid_range):
    return (np.arange(midprice-grid_range, midprice+grid_range, grid_distance))

grid = generate_grid(midprice=midprice, grid_distance=grid_distance, grid_range=350)
grid


array([ 870,  875,  880,  885,  890,  895,  900,  905,  910,  915,  920,
        925,  930,  935,  940,  945,  950,  955,  960,  965,  970,  975,
        980,  985,  990,  995, 1000, 1005, 1010, 1015, 1020, 1025, 1030,
       1035, 1040, 1045, 1050, 1055, 1060, 1065, 1070, 1075, 1080, 1085,
       1090, 1095, 1100, 1105, 1110, 1115, 1120, 1125, 1130, 1135, 1140,
       1145, 1150, 1155, 1160, 1165, 1170, 1175, 1180, 1185, 1190, 1195,
       1200, 1205, 1210, 1215, 1220, 1225, 1230, 1235, 1240, 1245, 1250,
       1255, 1260, 1265, 1270, 1275, 1280, 1285, 1290, 1295, 1300, 1305,
       1310, 1315, 1320, 1325, 1330, 1335, 1340, 1345, 1350, 1355, 1360,
       1365, 1370, 1375, 1380, 1385, 1390, 1395, 1400, 1405, 1410, 1415,
       1420, 1425, 1430, 1435, 1440, 1445, 1450, 1455, 1460, 1465, 1470,
       1475, 1480, 1485, 1490, 1495, 1500, 1505, 1510, 1515, 1520, 1525,
       1530, 1535, 1540, 1545, 1550, 1555, 1560, 1565])

In [26]:
signal = [0]*len(dataF)
i=0
for index, row in dataF.iterrows():
    for p in grid:
        if min(row.Low, row.High)<p and max(row.Low, row.High)>p:
            signal[i]=1
    i+=1

In [27]:
len(dataF)

51125

In [28]:
dataF["signal"]=signal
dataF[dataF["signal"]==1]

,Open,High,Low,Close,Volume,signal
Date,,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910,1
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670,1
2020-11-02 09:55:00,898.7,901.4,898.6,901.3,3705,1
2020-11-02 10:05:00,900.0,900.4,899.5,899.5,1044,1
2020-11-02 10:15:00,899.6,901.5,899.6,901.2,2305,1
...,...,...,...,...,...,...
2024-11-06 13:25:00,1328.9,1331.2,1328.5,1331.0,6002,1
2024-11-06 13:30:00,1331.1,1331.9,1329.9,1331.2,8046,1
2024-11-06 13:55:00,1331.6,1336.4,1331.3,1336.1,13116,1


In [29]:
dfpl = dataF[:].copy()
def SIGNAL():
    return dfpl.signal
dfpl['ATR'] = ta.atr(high = dfpl.High, low = dfpl.Low, close = dfpl.Close, length = 16)
dfpl.dropna(inplace=True)

In [30]:
from backtesting import Strategy
from backtesting import Backtest
import backtesting

class MyStrat(Strategy):
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.5*grid_distance #5*self.data.ATR[-1]
        TPSLRatio = 0.5

        if self.signal1==1 and len(self.trades)<=10000:
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1)

            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1)

bt = Backtest(dfpl, MyStrat, exclusive_orders=False)
stat = bt.run()
stat

Start                     2020-11-02 10:20:00
End                       2024-11-06 14:45:00
Duration                   1465 days 04:25:00
Exposure Time [%]                   89.909801
Equity Final [$]                      4997.65
Equity Peak [$]                      10072.05
Return [%]                           -50.0235
Buy & Hold Return [%]               49.115979
Return (Ann.) [%]                  -15.931252
Volatility (Ann.) [%]               17.421779
Sharpe Ratio                        -0.914445
Sortino Ratio                       -1.081327
Calmar Ratio                        -0.306021
Max. Drawdown [%]                  -52.059412
Avg. Drawdown [%]                  -17.385976
Max. Drawdown Duration     1465 days 01:20:00
Avg. Drawdown Duration      488 days 08:34:00
# Trades                                 5867
Win Rate [%]                        64.734958
Best Trade [%]                       3.088194
Worst Trade [%]                     -3.042455
Avg. Trade [%]                    

In [31]:
backtesting.set_bokeh_output(notebook=False)
bt.plot(show_legend=False, plot_width=None, plot_equity=True, plot_return=False,
plot_pl=False, plot_volume=False, plot_drawdown=False, smooth_equity=False, relative_equity=True,
superimpose=True, resample=False, reverse_indicators=False, open_browser=True)

GridPlot(id='p1444', ...)

In [32]:
stat._trades.sort_values(by="EntryBar").tail(20)

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
5847,-3,50565,50566,1350.0,1346.35,10.95,0.002704,2024-10-23 10:25:00,2024-10-23 10:30:00,None,0 days 00:05:00
5848,-3,50566,50638,1348.7,1344.75,11.85,0.002929,2024-10-23 10:30:00,2024-10-24 13:40:00,None,1 days 03:10:00
5849,-3,50639,50644,1345.5,1341.75,11.25,0.002787,2024-10-24 13:45:00,2024-10-24 14:10:00,None,0 days 00:25:00
5850,-3,50645,50647,1342.5,1338.55,11.85,0.002942,2024-10-24 14:15:00,2024-10-24 14:25:00,None,0 days 00:10:00
5851,-3,50648,50673,1337.0,1333.25,11.25,0.002805,2024-10-24 14:30:00,2024-10-25 10:55:00,None,0 days 20:25:00
5852,-3,50676,50687,1336.8,1333.15,10.95,0.002730,2024-10-25 11:10:00,2024-10-25 13:30:00,None,0 days 02:20:00
5853,-3,50687,50697,1333.8,1330.05,11.25,0.002812,2024-10-25 13:30:00,2024-10-25 14:20:00,None,0 days 00:50:00
5854,-3,50698,50758,1328.8,1336.60,-23.40,-0.005870,2024-10-25 14:25:00,2024-10-29 09:30:00,None,3 days 19:05:00
5855,-3,50773,50788,1340.1,1336.35,11.25,0.002798,2024-10-29 10:45:00,2024-10-29 13:25:00,None,0 days 02:40:00
5856,-3,50799,50825,1340.4,1336.45,11.85,0.002947,2024-10-29 14:20:00,2024-10-30 10:50:00,None,0 days 20:30:00
